[參考](https://ithelp.ithome.com.tw/m/articles/10264243)

# Content-Encoder

In [1]:
import tensorflow as tf
def Encoder(input_shape,dim_neck = 32, dim_emb =256,freq =22):
    initializer = tf.keras.initializers.GlorotUniform(tf.sqrt(2.0))
    
    inp = Input(shape=input_shape)
    ### 要把 336 的那維變成 512
    ####
    x= tf.transpose(inp,(0,2,1))
    x= Conv1D(512,kernel_size = 5, strides=1, padding="same",dilation_rate=1,kernel_initializer=initializer)(x)
    x = tf.transpose(x,(0,2,1))
    x = BatchNormalization()(x)
    x = ReLU()(x)
    ####
    x= tf.transpose(inp,(0,2,1))
    x= Conv1D(512,kernel_size = 5, strides=1, padding="same",dilation_rate=1,kernel_initializer=initializer)(x)
    x = tf.transpose(x,(0,2,1))
    x = BatchNormalization()(x)
    x = ReLU()(x)
    ####
    x= tf.transpose(inp,(0,2,1))
    x= Conv1D(512,kernel_size = 5, strides=1, padding="same",dilation_rate=1,kernel_initializer=initializer)(x)
    x = tf.transpose(x,(0,2,1))
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    ######################
    # 對 176 那維做 LSTM  #
    ######################
    
    x = tf.transpose(x,(0,2,1))
    lstm_tf_1 = LSTM(32,return_sequences = True)
    lstm_tf_2 = LSTM(32,return_sequences = True)
    
    x = Bidrectional(lstm_tf_1)(x)
    x = Bidrectional(lstm_tf_2)(x)
    
    ## 採樣
    x_up = x[:,:,:dim_neck]
    x_down = x[:,:,dim_neck:]
    codes = []
    
    for i in range(0,LEN_CROP, FREQ):
        codes.append(tf.concat((x_up[:,i+freq-1,:],x_down[:,i,:]),axis=-1))
        
    return Model(inputs=inp, outputs=codes, name="content_encoder")
    

# Decoder

In [2]:
def Decoder(encoder_input_shape):
    initializer = tf.keras.initializers.GlorotUniform(tf.sqrt(2.0))
    inputs = Input(shape = encoder_input_shape)
    
    # 進 LSTM 時 shape = (2,176,320)
    x = LSTM(512,return_sequences = True,kernel_initializer=initializer)(inputs)
    
    
    """
    3 個 5x1 Conv + BN + ReLU
    """
    x = Conv1D(512, kernel_size=5, strides=1, padding="same",dilation_rate=1,kernel_initializer=initializer)(x)
    x = tf.transpose(x,(0,2,1))
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = tf.transpose(x,(0,2,1))
    x = Conv1D(512,kernel_size=5, strides=1, padding="same",dilation_rate=1,kernel_initializer=initializer)(x)
    x = tf.transpose(x,(0,2,1))
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = tf.transpose(x,(0,2,1))
    
    ###########################
    # 對 512 那維做 LSTM      #
    ###########################
    
    x = LSTM(1024, return_sequences = True, kernel_initializer=initializer)(x)
    x = LSTM(1024, return_sequences = True, kernel_initionalizer=inializer)(x)
    
    """
    Linear
    """
    
    x= Dense(80)(x)
    
    return Model(inputs=inputs, output=x, name="decoder")
    

# Decoder 的第二個輸出

In [3]:
def Posnet(input_shape):
    initializer = tf.keras.initializers.GlorotUniform(1)
    inp = Input(shape = input_shape)
    
    """
    這裡是第二個輸出
    要把 80 的那維變成 512
    """
    
    """
    4 個 5x1 Conv + BN + ReLU
    """
    
    x = Conv1D(512, kernel_size=5, strides=1, padding="same", dilation_rate=1,kernel_initializer=intializer)(inp)
    x = tf.transpose(x,(0,2,1))
    x = BatchNormalization()(x)
    x = Activation("tanh")(x)
    
    x = tf.transpose(x,(0,2,1))
    x = Conv1D(512,kernel_size=5, strides=1,padding="same",dilation_rate=1,kernel_initializer=initializer)(x)
    x = tf.transpose(x,(0,2,1))
    x = BatchNormalization()(x)
    x = Activation('tanh')(x)
    
    x = tf.transpose(x,(0,2,1))
    x = Conv1D(512, kernel_size=5, strides=1,padding="same", dilation_rate=1, kernel_initializer=initializer)(x)
    x = tf.transpose(x,(0,2,1))
    
    x = BatchNormal()(x)
    x = Activation('tanh')(x)
    
    x = tf.transpose(x,(0,2,1))
    x = Conv1D(80, kernel_size = 5, strides=1, padding="same",dilation_ratew=1,kernel_initiaizer=initializer)(x)
    x = tf.transpose(x,(0,2,1))
    
    x = BatchNormalization()(x)
    
    return Model(inputs= inp, output= x, name= "posnet")

# 把上面三個組合起來, 這邊對應的是 model_vc 裡的 Generator

In [4]:
class AutoVC(tf.keras.Model):
    def __init__(self,dim_neck=32, dim_emb=256,len_crop=176,freq =22):
        super(AutoVC,self).__init__()
        self.encoder = Encoder((dim_emb+80,len_crop),dim_neck,dim_emb,freq)
        self.decoder = Decoder((len_crop,320))
        self.postnet = Posnet((len_crop,80))
        
    def call(self,inputs):
        x = inputs[0]
        c_org = inputs[1]
        c_trg = inputs[-1]
        batch_size = tf.shape(x)[0]
        
        x = tf.transpose(x,(0,2,1))
        c_org = tf.expand_dims(c_org,axis=1)
        c_org = tf.transpose(tf.broadcast_to(c_org(tf.shape(c_org)[0],LEN_CROP,tf.shape(c_org)[-1])),(0,2,1))
        # concat 80 那維
        x = tf.concat([x,c_org],axis=1)
        
        codes = self.encoder(x)
        if c_trg is None:
            return tf.concat(codes,axis=-1)
        
        tmp = []
        for code in codes:
            tc = tf.expand_dim(code,axis=1)
            tmp.append(tf.broadcast_to(tc,(batch_size,int(LEN_CROP/len(codes)),64)))
        code_exp = tf.concat(tmp,axis=1)
        
        c_trg = tf.expand_dims(c_trg,axis=1)
        c_trg = tf.broadcast_to(c_trg,(batch_size,tf.shape(x)[-1],DIM_EMB))
        
        # concat 64 那維
        encoder_outputs = tf.concat((code_exp,c_trg), axis=-1)
        mel_outputs = self.decoder(encoder_ouputs)
        
        mel_outputs_postnet = self.postnet(mel_outputs)
        mel_outputs_postnet = tf.transpose(mel_outputs_postnet,(0,2,1))
        mel_outputs_postnet = mel_outputs + mel_outputs_postnet
        
        return mel_outputs, mel_outputs_postnet, tf.concat(codes,axis=-1)

# 生成 D_VECTOR

下載 [Pre_train Model](https://drive.google.com/file/d/1ORAeb4DlS_65WDkQN6LHx5dPyCM5PAVV/view) 後定義 D_VECTOR, 這裡用的是 LSTM版的

In [12]:
import tensorflow.nn as nn

class D_VECTOR(tf.keras.layers.Layer):
    """ D_VECTOR speaker embedding."""
    
    def __init__(self, num_layers=3, dim_input=40, dim_cell=256, dim_emb=64):
        super(D_VECTOR, self).__init__()
        self.lstm = tf.keras.layers.LSTM(input_size=dim_input, hidden_size=dim_cell,
                           num_layers=num_layers, batch_first=True)
        self.embedding = tf.keras.layers.Linear(dim_cell, dim_emb)
        
    def forward(self, x):
        self.lstm.flatten_parameters()
        lstm_out, _ = self.lstm(x)
        embeds = self.embedding(lstm_out[:,-1,:])
        norm = embeds.norm(p=2, dim=-1, keepdim=True)
        embeds_normalized = embeds.div(norm)
        return embeds_normalized

In [13]:
import os
import pickle
# from model_bl import D_VECTOR
from collections import OrderedDict
import numpy as np
import torch

C = D_VECTOR(dim_input=80, dim_cell=768, dim_emb=256).eval().cpu()
c_checkpoint = torch.load('3000000-BL.ckpt',map_location=torch.device('cpu'))
new_state_dict = OrderedDict()

for key, val in c_checkpoint['model_b'].items():
    new_key = key[7:]
    new_state_dict[new_key] = val
    
C.load_state_dict(new_state_dict)

# 指的是說一個語者說了幾種不同內容的話, 讓資料的數量盡量一樣, 內容可以不一樣
num_uttrs = 68
len_crop = 176

# Directory containing mel-spectrograms
rootDir = './spmel'
dirName, subdirList, _ = next(os.walk(rootDir))
print('Found directory: %s' % dirName)

def pad_along_axis(array: np.ndarray, target_length: int, axis: int=0):
    pad_size = target_length - array.shape[axis]
    
    if pad_size <= 0:
        return array
    npad = [(0,0)] * array.ndim
    npad[axis] = (0,pad_size)
    
    return np.pad(array, pad_size)

speakers = []
for speaker in sorted(subdirList[1:]):
    print('Processing speaker: %s' % speaker)
    utterances = []
    utterances.append(speaker)
    _, _, fileList = next(os.walk(os.path.join(dirName,speaker)))
    fileList = fileList[:num_uttrs]
    
    # make speaker embedding
    assert len(fileList) >= num_uttrs
    idx_uttrs = np.random.choice(len(fileList), size=num_uttrs, replace=False)
    embs = []

    for i in range(num_uttrs):
        tmp = np.load(os.path.join(dirName, speaker, fileList[idx_uttrs[i]]),allow_pickle=True)
        
        # pad if the current one is too short
        if tmp.shape[0] <= len_crop:
            pad =int(len_crop - tmp.shape[0])
            tmp = pad_along_axis(tmp,pad)
            melsp = tf.convert_to_tensor(tmp[np.newaxis,:, :])
        else:
            left = np.random.randint(0, tmp.shape[0]-len_crop)
            melsp = tf.convert_to_tensor(tmp[np.newaxis, left:left+len_crop, :])
        
        emb = C(melsp)
        embs.append(emb,detach().squeeze.cpu().numpy())
        
utterance.append(np.mean(embs,axis=0))
for fileName in sorted(fileList):
    utterances.append(os.path.join(speaker,fileName))
speakers.append(utterance)

with open(os.path.join(rootDir,'train.pkl'),'wb') as handle:
    pickle.dump(speakers,handle)

TypeError: __init__() missing 1 required positional argument: 'units'

# 製作 Dataset

In [20]:
from torch.utils import data
import torch
import numpy as np
import pickle
import os
from multiprocessing import Process, Manager

class Utterances(data.Dataset):
    """Data class for the Utterances dataset"""
    
    def __init__(self,root_dir,len_crop):
        """Initialize and preprocess the Utterances dataset."""
        
        self.root_dir = root_dir
        self.len_crop = len_crop
        self.step = 10
        
        metaname = self.root_dir+"/train.pkl"
        print(metaname)
        meta = pickle.load(open(metaname,'rb'))
        
        """Load data using multiprocessing"""
        manager = Manager()
        meta = manager.list(meta)
        dataset = manager.list(len(meta)*[None])
        processes = []
        
        for i in range(0,len(meta), self.step):
            p = Process(target=self.load_data,
                        args =(meta[i:i+self.step],dataset,i))
            p.start()
            processes.append(p)
            
        for p in processes:
            p.join()
            
        self.train_dataset = list(dataset)
        self.num_tokens = len(self.train_dataset)
        
        print('Finished loading the dataset...')
        
    def load_data(self,submeta,dataset,idx_offset):
        for k, sbmt in enumerate(submeta):
            uttrs = len(sbmt)*[None]
            for j, tmp in enumerate(sbmt):
                if j<2: # fill in speaker id and embedding
                    uttrs[j] = tmp
                else: # load the mel-spectrograms
                    uttrs[j] = np.load(os.path.join(self.root_dir,tmp))
            dataset[idx_offset+k] = uttrs
            
    def __getitem__(self,index):
        #pick a random speaker
        dataset = self.train_dataset
        list_uttrs =dataset[index]
        emb_org = list_uttrs[1]
        
        # pick random uttr with random crop
        a = np.random.randint(2,len(list_uttrs))
        tmp = list_uttrs[a]
        
        if tmp.shape[0]<self.len_crop:
            len_pad = self.len_crop - tmp.shape[0]
            uttr = np.pad(tmp, ((0,len_pad),(0,0)),'constant')
        elif tmp.shape[0]> self.len_crop:
            left = np.random.randint(tmp.shape[0]-self.len_crop)
            uttr = tmp[left:left+self.len_crop, :]
        else:
            uttr = tmp
            
        return uttr, emb_org
    
    def __len__(self):
        """Return the number of speakers."""
        return self.num_tokens
    
def get_loader(root_dir,batch_size=2, len_crop=176,num_workers=0):
    """Build and return a data loader."""
    
    dataset = Utterances(root_dir,len_crop)
    
    worker_init_fn = lambda x: np.random.seed((torch.initial_seed()) % (2**32))
    data_loader = data.DataLoader(dataset=dataset,
                                  batch_size = batch_size,
                                  shuffle=True,
                                  num_workers = num_workers,
                                  drop_last=True,
                                  worker_init_fn = worker_init_fn)
    return data_loader

# Train-Loop

* 驗證的話至少要訓練 30000 次才聽得出來, 在 2080Ti 要跑約 6 小時, 要聽到不錯的結果要訓練 1000000次 , loop 會收斂到 0.0001


Tensorflow 版的要 DIM_NECK 設 44 才會 work , 目前原因不明

In [21]:

BATCH_SIZE = 100
LEN_CROP = 50
vcc_loader = get_loader('./spmel',BATCH_SIZE,LEN_CROP)


def train(step=30000, batch_size =2):
    print('....Strat....')
    
    for j in range(step):
        
        # 這裡的跟之前 pytorch 那邊一樣, 所以要確保輸入跟 pytorch 一樣
        try:
            x_real, emb_org = next(data_iter)
        except:
            data_iter = iter(vcc_loader)
            x_real, emb_org = next(data_iter)
            
        # 因為輸入資料是 torch tensor 記得要轉回 np
        x_real = x_real.detach().cpu().numpy().astype(np.float32)
        emb_org = emb_org.detach().cpu().numpy().astype(np.float32)
        
        # train_step 見下方
        g_loss_id, g_loss_id_psnt, g_loss_cd = train_step(x_real,emb_org,emb_org)
        
        if (j+1)%10 ==0:
            print(f"Step:{j}")
            print(f"G_loss_id:{g_loss_id}")
            print(f"G_loss_id_psnet:{g_loss_id_psnt}")
            print(f"G_loss_cd:{g_loss_cd}")
        if (j+2)%10 ==0:
            clear_output(wait=True)
        
        # 什麼時候存都可以, 使用的時候就 autovc.encoder.load_weight('encoder_weights') 載入就好
        
        if (j+1)%10000==0:
            autovc.encoder.save_weights(f"model/encoder_weights_step_{j+1}.h5")
            autovc.decoder.save_weights(f"model/decoder_weights_step_{j+1}.h5")
            autovc.postnet.save_weights(f"model/postnet_weights_step_{j+1}.h5")


./spmel/train.pkl


FileNotFoundError: [Errno 2] No such file or directory: './spmel/train.pkl'

# Train-Step

In [6]:
autovc_optimizer = tf.keras.optimizers.Adam(0.0001)

@tf.function
def train_step(x_real,emb_org,emb_trg):
    # tf.GradientTape() 等價於 loss.backward()
    
    with tf.GradientTape() as autovc_tape:
        x_identic, x_identic_psnt, code_real = autovc([x_real,emb_org,emb_trg])
        
        # loss 請參考昨天那篇
        g_loss_id = mse_loss(x_real, x_identic)
        g_loss_id_psnt = mse_loss(x_real, x_identic_psnt)
        
        code_reconst = autovc([x_identic_psnt, emb_org, None])
        
        g_loss_cd = l1_loss(code_real, code_reconst)
        g_loss = g_loss_id + g_loss_id_psnt + g_loss_cd
        
    gradients_of_autovc = autovc_tape.gradient(g_loss,autovc.trainable_variables)
    autovc_optimizer.apply_gradients(zip(gradients_of_autovc.trainable_variable))
    
    return g_loss_id, g_loss_id_psnt, g_loss_cd

In [7]:
# 接下來即可開始訓練
# 到這邊我們已經把 AutoVC 做過兩遍了，TF 做出來的效果跟 Pytorch 的是一樣的，只是 dim_neck 這個參數比較令人疑惑，為什麼 pytorch 的可以在 32 上成功但 TF 的不行，但兩邊在 freq = 22, dim_neck = 44 的情況下轉出來的聲音效果我聽起來是差不多的。

train(900000)

....Strat....


NameError: name 'vcc_loader' is not defined